# NFL Fantasy Football Projection Model Using XGBoost

This notebook presents a comprehensive workflow for building an NFL fantasy football projection model using XGBoost. Data sources include Yahoo, Pro-Football Reference, SportsDataIO, and NFLVerse. The goal is to leverage advanced machine learning techniques and rich datasets to generate accurate player projections for fantasy football analysis and decision-making.

In [1]:
import pandas as pd
import numpy as np
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print(f"Project Root: {project_root}")
print("Sys Path Before:", sys.path)
if project_root not in sys.path:
    print("Inserting project root to sys.path")
    sys.path.insert(0, project_root)

# Now import
from data_api import SportsDataIO, Yahoo, PFR
import utils
import yahoo_helpers

from dotenv import load_dotenv
load_dotenv()

Project Root: c:\Users\bengu\Documents\Sports Analysis Project\clairvoyent-raven-sports-analysis\src
Sys Path Before: ['C:\\Users\\bengu\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'C:\\Users\\bengu\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'C:\\Users\\bengu\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'C:\\Users\\bengu\\AppData\\Local\\Programs\\Python\\Python310', 'c:\\Users\\bengu\\.virtualenvs\\cfeproj-oIABPDjj', '', 'c:\\Users\\bengu\\.virtualenvs\\cfeproj-oIABPDjj\\lib\\site-packages', 'c:\\Users\\bengu\\.virtualenvs\\cfeproj-oIABPDjj\\lib\\site-packages\\win32', 'c:\\Users\\bengu\\.virtualenvs\\cfeproj-oIABPDjj\\lib\\site-packages\\win32\\lib', 'c:\\Users\\bengu\\.virtualenvs\\cfeproj-oIABPDjj\\lib\\site-packages\\Pythonwin']
Inserting project root to sys.path


True

## Import and clean data from nflverse source

In [2]:
NFLVERSE_DATA_PATH = r"C:\Users\bengu\Documents\Sports Analysis Project\clairvoyent-raven-sports-analysis\data\nfl_player_stats_cleaned.csv"
NFLVERSE_TEAMS_DATA_PATH = r"C:\Users\bengu\Documents\Sports Analysis Project\clairvoyent-raven-sports-analysis\data\nfl_team_stats.xlsx"
NFLVERSE_INJURIES_PATH = r"C:\Users\bengu\Documents\Sports Analysis Project\clairvoyent-raven-sports-analysis\data\nfl_injuries.xlsx"

all_players_df = pd.read_csv(NFLVERSE_DATA_PATH)
all_teams_df = pd.read_excel(NFLVERSE_TEAMS_DATA_PATH, engine="openpyxl")
injuries_df = pd.read_excel(NFLVERSE_INJURIES_PATH)

In [3]:
column_categories = {
    "standard": {
        "player_id",
        "player_name",
        "position",
        "position_group",
        "season",
        "week",
        "season_type",
        "team",
        "opponent_team",
        "pacr",
    },
    "passing": {
        "completions",
        "attempts",
        "passing_yards",
        "passing_tds",
        "passing_interceptions",
        "sacks_suffered",
        "sack_yards_lost",
        "sack_fumbles",
        "sack_fumbles_lost",
        "passing_air_yards",
        "passing_yards_after_catch",
        "passing_first_downs",
        "passing_epa",
        "passing_cpoe",
        "passing_2pt_conversions",
        "carries", # Include rushing stats for QBs
        "rushing_yards",
        "rushing_tds",
        "rushing_fumbles",
        "rushing_fumbles_lost",
        "rushing_first_downs",
        "rushing_epa",
        "rushing_2pt_conversions",
    },
    "rushing_and_receiving": {
        "carries",
        "rushing_yards",
        "rushing_tds",
        "rushing_fumbles",
        "rushing_fumbles_lost",
        "rushing_first_downs",
        "rushing_epa",
        "rushing_2pt_conversions",
        "receptions",
        "targets",
        "receiving_yards",
        "receiving_tds",
        "receiving_fumbles",
        "receiving_fumbles_lost",
        "receiving_air_yards",
        "receiving_yards_after_catch",
        "receiving_first_downs",
        "rushing_epa",
        "rushing_2pt_conversions",
        "receptions",
        "targets",
        "receiving_yards",
        "receiving_tds",
        "receiving_fumbles",
        "receiving_fumbles_lost",
        "receiving_air_yards",
        "receiving_yards_after_catch",
        "receiving_first_downs",
        "receiving_epa",
        "receiving_2pt_conversions",
        "racr",
        "target_share",
        "air_yards_share",
        "wopr",
    },
    "defense": {
        "special_teams_tds",
        "def_tackles_solo",
        "def_tackles_with_assist",
        "def_tackle_assists",
        "def_tackles_for_loss",
        "def_tackles_for_loss_yards",
        "def_fumbles_forced",
        "def_sacks",
        "def_sack_yards",
        "def_qb_hits",
        "def_interceptions",
        "def_interception_yards",
        "def_pass_defended",
        "def_tds",
        "def_fumbles",
        "def_safeties",
        "misc_yards",
        "fumble_recovery_own",
        "fumble_recovery_yards_own",
        "fumble_recovery_opp",
        "fumble_recovery_yards_opp",
        "fumble_recovery_tds",
        "penalties",
    },
    "kicking": {
        "fg_made",
        "fg_att",
        "fg_missed",
        "fg_blocked",
        "fg_long",
        "fg_pct",
        "fg_made_0_19",
        "fg_made_20_29",
        "fg_made_30_39",
        "fg_made_40_49",
        "fg_made_50_59",
        "fg_made_60_",
        "fg_missed_0_19",
        "fg_missed_20_29",
        "fg_missed_30_39",
        "fg_missed_40_49",
        "fg_missed_50_59",
        "fg_missed_60_",
        "fg_made_list",
        "fg_missed_list",
        "fg_blocked_list",
        "fg_made_distance",
        "fg_missed_distance",
        "fg_blocked_distance",
        "pat_made",
        "pat_att",
        "pat_missed",
        "pat_blocked",
        "pat_pct",
        "gwfg_made",
        "gwfg_att",
        "gwfg_missed",
        "gwfg_blocked",
        "gwfg_distance",
    },
    "special_teams": {
        "punt_returns",
        "punt_return_yards",
        "kickoff_returns",
        "kickoff_return_yards",
    },
}

categories_positions = {
    "passing": ["QB"],
    "rushing_and_receiving": ["RB", "WR"],
    # Not available "kicking": ["K"] 
}


In [4]:
# Establish dataframes for each positional group, filter by position value
def filter_by_positional_group(df, category_key_a, category_key_b="standard") -> pd.DataFrame:
    new_df = df.loc[:, list(column_categories[category_key_a] | column_categories[category_key_b])]
    new_df = new_df[new_df["position"].isin(categories_positions[category_key_a])]
    return new_df.reset_index(drop=True)

# Data of interest
passing_df = filter_by_positional_group(all_players_df, "passing")
rushing_and_receiving_df = filter_by_positional_group(all_players_df, "rushing_and_receiving")

# Save in case
defense_df = all_players_df.loc[:, list(column_categories["standard"] | column_categories["defense"])]
kicking_df = all_players_df.loc[:, list(column_categories["standard"] | column_categories["kicking"])]
specials_teams_df = all_players_df.loc[:, list(column_categories["standard"] | column_categories["special_teams"])]

# Give the dataframes each an alias
passing_df.alias = "passing_df"
rushing_and_receiving_df.alias = "rushing_and_receiving_df"
defense_df.alias = "defense_df"
kicking_df.alias = "kicking_df"
specials_teams_df.alias = "specials_teams_df"

In [5]:
for df in [passing_df, rushing_and_receiving_df, defense_df, kicking_df, specials_teams_df]:
    null_values = 0
    print(f"Checking {df.alias} for null values")
    null_counts = df.isna().sum()

    for column, count in zip(null_counts.index, null_counts):
        if count != 0:
            null_values += 1
            print(f"Column: {column} contains {count} null values.")
    
    print(f"{df.alias} contains {null_values} null values.")

df = None

Checking passing_df for null values
passing_df contains 0 null values.
Checking rushing_and_receiving_df for null values
rushing_and_receiving_df contains 0 null values.
Checking defense_df for null values
defense_df contains 0 null values.
Checking kicking_df for null values
kicking_df contains 0 null values.
Checking specials_teams_df for null values
specials_teams_df contains 0 null values.


### Implement and Apply Model

Implement multiple linear regression models to predict multiple statistics for rushers and receivers. 

* Define the targets: rushing_yards, rushing_tds, receiving_yards, receiving_tds

In [6]:
rushing_and_receiving_df.columns

Index(['receiving_fumbles_lost', 'rushing_fumbles_lost',
       'receiving_yards_after_catch', 'player_name', 'targets', 'rushing_epa',
       'receiving_epa', 'receiving_2pt_conversions', 'position_group',
       'rushing_yards', 'receiving_yards', 'carries',
       'rushing_2pt_conversions', 'position', 'target_share',
       'rushing_fumbles', 'receptions', 'receiving_tds', 'opponent_team',
       'racr', 'pacr', 'rushing_first_downs', 'receiving_fumbles',
       'air_yards_share', 'rushing_tds', 'season_type', 'season',
       'receiving_first_downs', 'week', 'team', 'player_id', 'wopr',
       'receiving_air_yards'],
      dtype='object')

In [7]:
"""
Collect xs, and ys.
Normalize data using Z-score normalization
(val - mean_val) / standard_deviation

Create 4 buckets, one for each statistical category.

Prefix   | Stat Category
-------------------------
rsh_yd_ -> rushing_yards
rsh_td_ -> rushing_tds
rc_yd_  -> receiving_yards
rc_td_  -> receiving_tds
"""

inputs = {
    "rsh_yd": ["rushing_tds", "rushing_yards", "rushing_fumbles_lost", "racr", "rushing_epa", "rushing_fumbles", "carries", "pacr", "rushing_first_downs", "rushing_2pt_conversions"],
    "rsh_td": ["rushing_tds", "rushing_yards", "rushing_fumbles_lost", "racr", "rushing_epa", "rushing_fumbles", "carries", "pacr", "rushing_first_downs", "rushing_2pt_conversions"],
    "rc_yd" : ["receiving_epa", "receiving_2pt_conversions", "racr", "air_yards_share", "receiving_tds", "receiving_first_downs", "wopr", "receiving_yards_after_catch", "receiving_air_yards", "target_share", "pacr", "targets", "receiving_yards",  "receiving_fumbles_lost", "receptions"],
    "rc_td" : ["receiving_epa", "receiving_2pt_conversions", "racr", "air_yards_share", "receiving_tds", "receiving_first_downs", "wopr", "receiving_yards_after_catch", "receiving_air_yards", "target_share", "pacr", "targets", "receiving_yards",  "receiving_fumbles_lost", "receptions"],
    "def": ["def_tackles_solo", "def_tackles_with_assist", "def_tackle_assists", "def_tackles_for_loss", "def_tackles_for_loss_yards", "def_fumbles_forced", "def_sacks", "def_sack_yards", "def_qb_hits", "def_interceptions", "def_interception_yards", "def_pass_defended", "def_tds", "def_fumbles", "def_safeties"]

}

# Stat 1 rushing_yards
rsh_yd_X = rushing_and_receiving_df[inputs["rsh_yd"] + ["season", "week", "player_id", "opponent_team"]]
# Stat 2 rushing_tds
rsh_td_X = rushing_and_receiving_df[inputs["rsh_td"] + ["season", "week", "player_id", "opponent_team"]]
# Stat 3 receiving_yards
rc_yd_X = rushing_and_receiving_df[inputs["rc_yd"] + ["season", "week", "player_id", "opponent_team"]]
# Stat 4 receiving_tds
rc_td_X = rushing_and_receiving_df[inputs["rc_td"] + ["season", "week", "player_id", "opponent_team"]]

# Defensive stats for opponent
defense_stats_df = all_teams_df[inputs["def"] + ["season", "week", "team"]]

In [8]:
# # Collect rolling data by season-week
# rsh_yd_X = rsh_yd_X.sort_values(["season", "week", "player_id"])

# # Group by season + player_id, then apply rolling mean over 3 weeks
# rsh_yd_X[[f"{c}_roll3_shift" for c in inputs["rsh_yd"]]] = (
#     rsh_yd_X.groupby(["season", "player_id"])[inputs["rsh_yd"]]
#       .transform(lambda x: x.rolling(3, min_periods=1).mean().shift(1))
# )

# # Collect the rolling defensive stats
# defense_stats_df = defense_stats_df.sort_values(["season", "week", "team"])

# defense_stats_df[[f"{c}_roll3_shift" for c in inputs["def"]]] = (
#     defense_stats_df.groupby(["season", "team"])[inputs["def"]]
#       .transform(lambda x: x.rolling(3, min_periods=1).mean().shift(1))
# )

In [9]:
def calculate_rolling_data(df: pd.DataFrame, sort_values: list, input_ref: str, groupby: list, rolling_period: int=3, min_periods: int=1, shift: int=1):
    df = df.sort_values(sort_values)

    df[[f"{c}_roll{rolling_period}_shift" for c in inputs[input_ref]]] = (
        df.groupby(groupby)[inputs[input_ref]]
         .transform(lambda x: x.rolling(rolling_period, min_periods=min_periods).mean().shift(shift))
    )

    return df

rsh_yd_X = calculate_rolling_data(rsh_yd_X, ["season", "week", "player_id"], "rsh_yd", ["season", "player_id"])
rsh_td_X = calculate_rolling_data(rsh_td_X, ["season", "week", "player_id"], "rsh_td", ["season", "player_id"])
rc_yd_X = calculate_rolling_data(rc_yd_X, ["season", "week", "player_id"], "rc_yd", ["season", "player_id"])
rc_td_X = calculate_rolling_data(rc_td_X, ["season", "week", "player_id"], "rc_td", ["season", "player_id"])
defense_stats_df = calculate_rolling_data(defense_stats_df, ["season", "week", "team"], "def", ["season", "team"])

In [10]:
# Normalize the data using Z-Score Standardization (sklearn StandardScaler)
from sklearn.preprocessing import StandardScaler

rsh_yd_input_cols = [col + "_roll3_shift" for col in inputs["rsh_yd"]]
rsh_td_input_cols = [col + "_roll3_shift" for col in inputs["rsh_td"]]
rc_yd_input_cols = [col + "_roll3_shift" for col in inputs["rc_yd"]]
rc_td_input_cols = [col + "_roll3_shift" for col in inputs["rc_td"]]
def_input_cols = [col + "_roll3_shift" for col in inputs["def"]]

scalers = {}

def scale_inplace(df, cols, name):
    # optionally: df = df.copy()  # if you want to avoid mutating the original
    scaler = StandardScaler()
    df.loc[:, cols] = scaler.fit_transform(df[cols])
    scalers[name] = scaler
    return df

rsh_yd_X = scale_inplace(rsh_yd_X, rsh_yd_input_cols, "rsh_yd")
rsh_td_X = scale_inplace(rsh_td_X, rsh_td_input_cols, "rsh_td")
rc_yd_X  = scale_inplace(rc_yd_X,  rc_yd_input_cols,  "rc_yd")
rc_td_X  = scale_inplace(rc_td_X,  rc_td_input_cols,  "rc_td")
defense_stats_df = scale_inplace(defense_stats_df, def_input_cols, "def")

In [11]:
# Merge the defensive dataframe to the offensive one.
rsh_yd_X = rsh_yd_X.merge(defense_stats_df, how="left", left_on=["opponent_team", "season", "week"], right_on=["team", "season", "week"])
rsh_td_X = rsh_td_X.merge(defense_stats_df, how="left", left_on=["opponent_team", "season", "week"], right_on=["team", "season", "week"])
rc_yd_X = rc_yd_X.merge(defense_stats_df, how="left", left_on=["opponent_team", "season", "week"], right_on=["team", "season", "week"])
rc_td_X = rc_td_X.merge(defense_stats_df, how="left", left_on=["opponent_team", "season", "week"], right_on=["team", "season", "week"])

### Apply the XGBoost Model

In [12]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import r2_score

model_results = {}
models = {}

input_dataframes = [rsh_yd_X, rsh_yd_X, rc_yd_X, rc_td_X]
inputs = [rsh_yd_input_cols, rsh_td_input_cols, rc_yd_input_cols, rc_td_input_cols]
targets = ["rushing_yards", "rushing_tds", "receiving_yards", "receiving_tds"]

for df, input, target in zip(input_dataframes, inputs, targets):
    X = df.dropna()[input + def_input_cols].reset_index(drop=True)
    y = df.dropna()[target].reset_index(drop=True)

    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model = xgb.XGBRegressor(
        n_estimators=2000,          # large cap; early stopping will find the best round
        learning_rate=0.03,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        reg_alpha=0.0,
        objective="reg:squarederror",
        tree_method="hist",         # fast/accurate; use "gpu_hist" if you have a GPU
        eval_metric="rmse",
        early_stopping_rounds=100,  # stops when no improvement
        random_state=42
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=False
    )

    # Evaluate
    preds = model.predict(X_valid)
    rmse = root_mean_squared_error(y_valid, preds)
    
    r2 = r2_score(y_valid, preds)

    model_results[target] = {"best_iteration": model.best_iteration, "validation_rmse": f"{rmse:.4f}", "r2": f"{r2:.3f}"}
    models[target] = model
    # # Feature importance (gain-based)
    # imp = (
    #     pd.Series(model.feature_importances_, index=model.get_booster().feature_names)
    #     .sort_values(ascending=False)
    # )
    # print(imp.head(15))

In [13]:
print(model_results)

{'rushing_yards': {'best_iteration': 196, 'validation_rmse': '19.1251', 'r2': '0.596'}, 'rushing_tds': {'best_iteration': 88, 'validation_rmse': '0.3197', 'r2': '0.204'}, 'receiving_yards': {'best_iteration': 135, 'validation_rmse': '26.2352', 'r2': '0.368'}, 'receiving_tds': {'best_iteration': 81, 'validation_rmse': '0.3920', 'r2': '0.099'}}


### Check Sample Predictions

In [ ]:
current_week = rsh_yd_X[(rsh_yd_X["season"] == 2025) & (rsh_yd_X["week"] == 4)].dropna().reset_index(drop=True)
current_week_X = current_week[rsh_yd_input_cols + def_input_cols].reset_index(drop=True)
preds = models["rushing_yards"].predict(current_week_X)

all_players_unique = all_players_df.drop_duplicates("player_id")
current_week = current_week.merge(all_players_unique[["player_name","player_id"]],
                                  how="left", on="player_id")

current_week["pred"] = preds

In [72]:
pd.set_option("display.max_rows", None)
print(current_week[["player_name", "rushing_yards", "pred"]])


           player_name  rushing_yards       pred
0            A.Thielen              0   0.651081
1              K.Allen              0   0.658451
2            D.Hopkins              0   0.666002
3              B.Cooks              0   0.644510
4              D.Adams              0   0.618229
5              S.Diggs              0   0.635193
6             D.Carter              0   0.836890
7            T.Lockett              0   0.647906
8            S.Shepard              6   0.675436
9              C.Moore              9   0.615775
10           K.Raymond              0   1.840903
11             D.Henry             42  58.005154
12              T.Hill              0   0.703644
13         C.McCaffrey             49  61.987526
14            K.Bourne              0   0.663959
15           T.Patrick              0   0.770357
16            S.Perine              4   8.134865
17           M.Hollins              0   0.625759
18             J.Agnew              0   0.865695
19             D.Moo